In [1]:
import pandas as pd
import numpy as np

import _pickle as pkl

# train_df=pd.read_csv('./complaints_train_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

from gensim.parsing.preprocessing import STOPWORDS
def remove_stopwords(tokens):
    # input and outputs a list of words
    return [word for word in tokens if word not in STOPWORDS]

train_df=pd.read_csv('./complaints_train_validation_data_clean.csv',usecols=["index","category_name","complaint_title","complaint_description",],na_filter=False)

whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

sentences = (train_df['complaint_title']+" "+train_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
train_df.insert(1,"complaint_text",sentences)


val_df=pd.read_csv('./complaints_validation_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

test_df=pd.read_csv('./complaints_test_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

## gold labels
gold_df=pd.read_csv('./clean-gold-labels.tsv',sep='\t',usecols=["category_name","complaint_description",],na_filter=False)
gold_df['complaint_title'] = gold_df['complaint_description']
sentences = (gold_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
gold_df.insert(1,"complaint_text",sentences)
######

sentences = (test_df['complaint_title']+" "+test_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
test_df.insert(1,"complaint_text",sentences)

gold_df.head()

category_name  \
0                                  Garbage   
1                         New Bus Shelters   
2                              Electricity   
3                                Bad Roads   
4  Land and Property Encroachment - Others   

                                      complaint_text  \
0  [garbage, collector, coming, collect, househol...   
1  [provide, bus, shelters, minimal, request, com...   
2  [days, complete, power, cut, annasandra, palya...   
3  [roads, dug, lay, drainage, pipes, laying, pip...   
4  [place, sites, connects, roads, sites, illegal...   

                               complaint_description  \
0  Garbage collector is not coming to collect hou...   
1  Please provide bus shelters. Its the most mini...   
2  2 days complete power cut in Annasandra Palya ...   
3  The roads were dug to lay the drainage pipes a...   
4  Next to my place there are 2 sites which conne...   

                                     complaint_title  
0  Garbage collector is not coming to collect hou...  
1  Please provide bus shelters. Its the most mini...  
2  2 days complete power cut in Annasandra Palya ...  
3  The roads were dug to lay the drainage pipes a...  
4  Next to my place there are 2 sites which conne...

In [2]:
colsize = len(train_df['category_name'])
train_df['category_name'] = train_df["category_name"].astype('category')
#train_df['true_label'] = pd.Series(np.zeros(colsize), index=train_df.index)
train_df['predicted_label'] = pd.Series(np.zeros(colsize), index=train_df.index)

# train_df['true_label'] = train_df['category_name'].cat.codes
# for i in range(colsize):
#     if(train_df['true_label'][i]==5):
#         print(train_df['true_label'][i],train_df['category_name'][i])

In [3]:
train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero})

complaint_title
category_name                                                      
Air Pollution                                                   187
Autorickshaws and Taxis                                         110
BMTC - Driver or Conductor                                      167
BMTC - Need new Bus Route                                       315
BMTC - Others                                                   826
Bad Roads                                                      5410
Broken Storm Water Drains                                       137
Cattle                                                           61
Clearing of Blockage of Under Ground Drainage P...              427
Desilting - Lakes                                                94
Diseases                                                        211
Electricity                                                    1120
Flooding of Roads and Footpaths                                 278
Footpaths                                                      1111
Garbage                                                        7492
Government Land Encroachment                                     83
Hawkers and Vendors                                             169
Hoardings                                                       201
Illegal posters and Hoardings                                    73
Lakes - Others                                                   80
Maintenance of Roads and Footpaths - Others                     210
Manholes                                                         88
Mosquitos                                                       129
Need New Streetlights                                           184
Need New Toilets                                                125
New Bus Shelters                                                608
No Sewage Drains                                                471
Noise Pollution                                                 249
Others                                                          655
Overflow of Storm Water Drains                                  239
Parking Violations                                              400
Parks and playgrounds                                           211
Potholes                                                       5347
Public Nuisance                                                 224
Repair of streetlights                                         1342
Sewage and Storm Water Drains - Others                          231
Stray Dogs                                                     3159
Traffic                                                        3973
Trees, Parks and Playgrounds - Others                            95
Unauthorized Construction                                        87
Water Leakage                                                   245
Water Supply                                                    771

In [4]:
for i,x in enumerate(train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index._data):
    print(i,x)

0 Air Pollution
1 Autorickshaws and Taxis
2 BMTC - Driver or Conductor
3 BMTC - Need new Bus Route
4 BMTC - Others
5 Bad Roads
6 Broken Storm Water Drains
7 Cattle
8 Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover
9 Desilting - Lakes
10 Diseases
11 Electricity
12 Flooding of Roads and Footpaths
13 Footpaths
14 Garbage
15 Government Land Encroachment
16 Hawkers and Vendors
17 Hoardings
18 Illegal posters and Hoardings
19 Lakes - Others
20 Maintenance of Roads and Footpaths - Others
21 Manholes
22 Mosquitos
23 Need New Streetlights
24 Need New Toilets 
25 New Bus Shelters
26 No Sewage Drains
27 Noise Pollution
28 Others
29 Overflow of Storm Water Drains
30 Parking Violations
31 Parks and playgrounds
32 Potholes
33 Public Nuisance
34 Repair of streetlights
35 Sewage and Storm Water Drains - Others
36 Stray Dogs
37 Traffic
38 Trees, Parks and Playgrounds - Others
39 Unauthorized Construction
40 Water Leakage
41 Water Supply 


In [5]:
class_names = train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index.tolist()

pkl.dump(class_names,open("class_names.p","wb"))

noOfClasses = len(train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index)

print(class_names)


['Air Pollution', 'Autorickshaws and Taxis', 'BMTC - Driver or Conductor', 'BMTC - Need new Bus Route', 'BMTC - Others', 'Bad Roads', 'Broken Storm Water Drains', 'Cattle', 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover', 'Desilting - Lakes', 'Diseases', 'Electricity', 'Flooding of Roads and Footpaths', 'Footpaths', 'Garbage', 'Government Land Encroachment', 'Hawkers and Vendors', 'Hoardings', 'Illegal posters and Hoardings', 'Lakes - Others', 'Maintenance of Roads and Footpaths - Others', 'Manholes', 'Mosquitos', 'Need New Streetlights', 'Need New Toilets ', 'New Bus Shelters', 'No Sewage Drains', 'Noise Pollution', 'Others', 'Overflow of Storm Water Drains', 'Parking Violations', 'Parks and playgrounds', 'Potholes', 'Public Nuisance', 'Repair of streetlights', 'Sewage and Storm Water Drains - Others', 'Stray Dogs', 'Traffic', 'Trees, Parks and Playgrounds - Others', 'Unauthorized Construction', 'Water Leakage', 'Water Supp

In [6]:
train_df.iloc[3]['complaint_title']+" "+train_df.iloc[3]['complaint_description']

'Road Condition of Ejipura after the sewage pipe line work Dear All,\\n\\nI am writing this email to you as a resident of Ejipura location in distress for more than 3 weeks.\\nI know for the fact that BBMP / BWSSB has taken initiative to lay new drainage pipe lines for most locations in Ejipura. However it looks like there is no progress at all.\\n\\nThe pipelines are layed, however the work is half done. No one really bothers how the residents manage the rest of the day and even in case of any emergency, we cannot take our car or ambulances inside. We have many senior citizens and children\\\'s living here and they have horrible time trying to cross these broken road & potholes. It\\\'s only matter of time when something bad happens & we are in the \\"TV News\\"!!\\n\\nI am sharing some of the pictures taken at Ejipura, Sri Rama Temple Road - 20th C Cross (beside ICICI ATM), the work is as it is and no one is really bothered to fix the problem. I have raised a complaint in both BWSSB 

In [8]:
keywords = {'Air Pollution':['dust|smoke|burn'],
 'Autorickshaws and Taxis':['taxi','auto','autorickshaw'],
 'BMTC - Driver or Conductor':['rude','behaviour'],
 'BMTC - Need new Bus Route':['frequency'],
 'BMTC - Others':[],
 'Bad Roads':['road'],
 'Broken Storm Water Drains':['overflow','drainage'],
 'Cattle':['cows','buffaloes','goats','cow'],
 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover':[],
 'Desilting - Lakes':['lake'],
 'Diseases':['malaria','dengue','cholera','fever','disease','hospital','epidemic'],
 'Electricity':['power','current','power cut'],
 'Flooding of Roads and Footpaths':['water','flood','floods'],
 'Footpaths':['footpath'],
 'Garbage':['waste','plastic','dirt'],
 'Government Land Encroachment':['occupy','illegal'],
 'Hawkers and Vendors':[],
 'Hoardings':['advertise'],
 'Illegal posters and Hoardings':['banner', 'ads ','advertise'],
 'Lakes - Others':['lake'],
 'Maintenance of Roads and Footpaths - Others':[],
 'Manholes':['manhole','man hole'],
 'Mosquitos':['mosquito','mosquitoe','mosquitoes','dengue','malaria'],
 'Need New Streetlights':['streetlight','light','new streetlight'],
 'Need New Toilets ':['toilet','urinal','urinate'],
 'New Bus Shelters':['shelter'],
 'No Sewage Drains':['drainage'],
 'Noise Pollution':['siren','speakers','speakers','loud'],
 'Others':[],
 'Overflow of Storm Water Drains':['pipes'],
 'Parking Violations':['parked','parker'],
 'Parks and playgrounds':['park','play','playground'],
 'Potholes':['holes','pothole'],
 'Public Nuisance':[],
 'Repair of streetlights':['streetlight','light','broken','damaged'],
 'Sewage and Storm Water Drains - Others':['drainage'],
 'Stray Dogs':['dog'],
 'Traffic':['vehicles'],
 'Trees, Parks and Playgrounds - Others':['tree'],
 'Unauthorized Construction':['encroach','building','built'],
 'Water Leakage':[],
 'Water Supply ':[]}

In [9]:
regexPatterns = {'Air Pollution':['air.*pollution|pollution|dust'],
 'Autorickshaws and Taxis':['autorickshaws|taxis|taxi|auto|autorickshaw'],
 'BMTC - Driver or Conductor':['bmtc.*driver|bmtc.*conductor|bus.*driver|bus.*conductor'],
 'BMTC - Need new Bus Route':['bus.*route'],
 'BMTC - Others':['bmtc'],
 'Bad Roads':['bad.*road|road.*bad'],
 'Broken Storm Water Drains':['(broken|damage).*(drain)'],
 'Cattle':['(cattle|cows|buffaloes|goats)'],
 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover':['clearing|blockage|under|ground|drainage|pipelines|replacement|damaged|missing|manhole|cover'],
 'Desilting - Lakes':['lake'],
 'Diseases':['diseases|malaria|dengue|cholera'],
 'Electricity':['electricity|power|current|power.*cut'],
 'Flooding of Roads and Footpaths':['((water|flood|flow).*(roads|footpaths))|((roads|footpaths).*(water|flood|flow))'],
 'Footpaths':['footpath'],
 'Garbage':['garbage|waste|plastic|dirt'],
 'Government Land Encroachment':['(government.*land).*(encroach|occupy|illegal)'],
 'Hawkers and Vendors':['(hawkers|vendors)'],
 'Hoardings':['(hoardings|advertisements)'],
 'Illegal posters and Hoardings':['posters|hoardings|banner|ads|advertise'],
 'Lakes - Others':['lake'],
 'Maintenance of Roads and Footpaths - Others':['(maintenance).*(roads|footpaths)'],
 'Manholes':['(manholes|manhole|man hole)'],
 'Mosquitos':['mosquito|mosquitoe|mosquitoes|dengue|malaria'],
 'Need New Streetlights':['(need|no|new).*(streetlight|light)'],
 'Need New Toilets ':['toilets|toilet|urinal|urinate'],
 'New Bus Shelters':['bus.*shelter|shelter.*bus'],
 'No Sewage Drains':['drain'],
 'Noise Pollution':['noise|noise.*pollution|siren|speakers|speakers|loud'],
 'Others':['others'],
 'Overflow of Storm Water Drains':['overflow.*(drains|pipes)'],
 'Parking Violations':['parking|parked|parker'],
 'Parks and playgrounds':['(parks|playgrounds|park|play|playground)'],
 'Potholes':['(pot hole|holes|pothole)'],
 'Public Nuisance':['(public.*nuisance|nuisance)'],
 'Repair of streetlights':['((light).*(repair|broke|damage))|((repair|broke|damage).*(light))'],
 'Sewage and Storm Water Drains - Others':['(sewage|storm|water|drains|drainage)'],
 'Stray Dogs':['(stray|dogs|dog)'],
 'Traffic':['(traffic|vehicles)'],
 'Trees, Parks and Playgrounds - Others':['(trees|parks|playgrounds|tree)'],
 'Unauthorized Construction':['encroach','building','built'],
 'Water Leakage':['water.*leak|leak.*water'],
 'Water Supply ':['water.*supply|supply.*water']}

In [10]:
extracts_df=pd.read_csv('./p.tsv',sep='\t',usecols=["category_name","Entity","complaint words",],na_filter=False)

extracts_df = extracts_df[extracts_df['category_name'].isin(class_names)]
# extracts_df
def combine(x):
    x = x.tolist()
    x = set(x)
    x = '|'.join(list(x)).lower()
    return x
extracts_df = extracts_df.groupby("category_name").agg({"Entity": combine,"complaint words":combine })
extracts_df.to_csv("extracts.csv")
extracts_df


Entity  \
category_name                                                                                           
Air Pollution                                                                             bus|garbage   
BMTC - Driver or Conductor                                                                       gbus   
BMTC - Need new Bus Route                                                                         bus   
BMTC - Others                                                                             bus|traffic   
Bad Roads                                                  roads|footpath|permit|pothole|roads|sewage   
Broken Storm Water Drains                                                                       drain   
Clearing of Blockage of Under Ground Drainage P...                                        tank|sewage   
Desilting - Lakes                                                                                lake   
Diseases                                                                                  garbage|cat   
Electricity                                          transformer|electricity|street light|electricity   
Flooding of Roads and Footpaths                                    water|roads|footpath|pothole|roads   
Footpaths                                                                      garbage|footpath|drain   
Garbage                                             clan|garbage|pothole|dump|roads|garbage|dustbi...   
Hawkers and Vendors                                                                          footpath   
Illegal posters and Hoardings                                                           trees|parking   
Lakes - Others                                                                                   lake   
Maintenance of Roads and Footpaths - Others                                                     roads   
Manholes                                                                                      manhole   
Mosquitos                                                                                  mosquitoes   
Need New Streetlights                                                             street light|lightd   
New Bus Shelters                                                                              bus|bus   
No Sewage Drains                                                           drain|garbage|roads|sewage   
Noise Pollution                                                                                   dog   
Others                                                      traffic|bus|footpath|drain|center|bribery   
Overflow of Storm Water Drains                                                           drain|sewage   
Parking Violations                                                             parking|footpath|roads   
Parks and playgrounds                                                                      trees|park   
Potholes                                                      roads|water|pothole|land|potholes|drain   
Public Nuisance                                                                      public|house|bar   
Repair of streetlights                                                                   street light   
Sewage and Storm Water Drains - Others                                                        manhole   
Stray Dogs                                                     dog|dog|dogs,light|stray dogs|hoarding   
Traffic                                             traffic|subway|skywalk|bus|roads|footpath|garb...   
Trees, Parks and Playgrounds - Others                                                            park   
Water Leakage                                                                                   water   

                                                                                      complaint words  
category_name                                                                                          
A

In [11]:
import re
class_words = [ re.sub('-','',x).lower().split() + keywords[x] for x in class_names ]
print(class_words,len(class_words))

[['air', 'pollution', 'dust|smoke|burn'], ['autorickshaws', 'and', 'taxis', 'taxi', 'auto', 'autorickshaw'], ['bmtc', 'driver', 'or', 'conductor', 'rude', 'behaviour'], ['bmtc', 'need', 'new', 'bus', 'route', 'frequency'], ['bmtc', 'others'], ['bad', 'roads', 'road'], ['broken', 'storm', 'water', 'drains', 'overflow', 'drainage'], ['cattle', 'cows', 'buffaloes', 'goats', 'cow'], ['clearing', 'of', 'blockage', 'of', 'under', 'ground', 'drainage', 'pipelines', 'and', 'replacement', 'of', 'damaged', 'or', 'missing', 'manhole', 'cover'], ['desilting', 'lakes', 'lake'], ['diseases', 'malaria', 'dengue', 'cholera', 'fever', 'disease', 'hospital', 'epidemic'], ['electricity', 'power', 'current', 'power cut'], ['flooding', 'of', 'roads', 'and', 'footpaths', 'water', 'flood', 'floods'], ['footpaths', 'footpath'], ['garbage', 'waste', 'plastic', 'dirt'], ['government', 'land', 'encroachment', 'occupy', 'illegal'], ['hawkers', 'and', 'vendors'], ['hoardings', 'advertise'], ['illegal', 'posters', 

In [12]:
for v in (train_df.loc[train_df['category_name'] =='Air Pollution']).iterrows():
    print(v[0])
#     print(v[1]['category_name'])
    print(v[1]['complaint_title'])
    print(v[1]['complaint_description'])

94
Smoking in Public Place
Many people are still smoking in public place, particularly in front of tea/bakery shops. Panathur Main Road, Munnireddy Layout Near SBM bank branch.
525
Cement piled up on the road. toxic to kids and elders in the neighbour hood
Cement is not stored in container n bags . it left out in open in huge quantity. can cause serious chronic lung diseases to babies and kids in nearby houses and apartments.
731
Fine Dust Behind Graphite India road
The strech between graphite India towards ORR thru big basket warehouse has air pollution.This stretch has lot of fine sand and dust  and when vehicles commute makes the road worst for the commuters to breath. Please take necessary action at earliest. Thanks
883
Pollution due to VMR granite factory
Dear Sir,\n\nI being a sufferer of a public nuisance am helpless to report an issue which is causing serious threat to my and my families\'s health on a regular basis and which is not taken care of seriously till now.\n\nAt 4:00 

In [9]:
## training word2vec 


# import gensim.matutils as gm

# from gensim.models.keyedvectors import KeyedVectors

# import gensim, logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
# model = gensim.models.Word2Vec(sentences, size=200, window=5, min_count=5)

# model.save("../word2vec_icmcData")

#to continue training
# model = Word2Vec.load("../word2vec_icmcData")

In [10]:
# print(len(model.wv.vocab))
# print([x for x in model.wv.vocab.keys()])
# model.accuracy('../questions-words.txt')



In [31]:
import gensim.matutils as gm
import gensim
from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
# model = KeyedVectors.load_word2vec_format('../glove_w2v.txt', binary=False)  # C binary format


model = gensim.models.Word2Vec.load("./J/word2vec_bangalore_description_train_min_count_5_clean.model")
model = model.wv

# model = KeyedVectors.load_word2vec_format('../word2vec_icmcData.txt', binary=False)  # C binary format
 
wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")



def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity 

In [14]:
import os
import re

stopwords_pattern = ' of| and| no| others| or| -|,|no '

def ltp(x):
    return '(' + '|'.join(x) + ')'

def create_LF_Based_On_Category_Name(debug=False):
    if os.path.exists("Category_Name_LFs.py"):
        os.remove("Category_Name_LFs.py")
        
    f = open("Category_Name_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])

        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
        pattern= re.sub(" $","",pattern)
        
        words = pattern.split(',')
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Category_Name_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return ('''+str(i)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()
    


    

def create_LF_Based_On_Keywords(debug=False):
    if os.path.exists("KeyWord_Based_LFs.py"):
        os.remove("KeyWord_Based_LFs.py")
    f = open("KeyWord_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_KeyWord_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return ('''+str(i)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()
    
def create_LF_Extracts_Phrases_Regex(debug=False):
    if os.path.exists("Regex_Based_Extracts_Phrases_LFs.py"):
        os.remove("Regex_Based_Extracts_Phrases_LFs.py")
    f = open("Regex_Based_Extracts_Phrases_LFs.py","a+")
    for i in range(len(class_names)):
        if(class_names[i] in extracts_df.index.tolist()):
            functionName = re.sub(r'( )+|-|,','',class_names[i])

            pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
            pattern= re.sub("( )+",",",pattern)

            words = pattern.split(',')
            ##### add keywords #####

            words = words+ keywords[class_names[i]]
            ####
            wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'


            if(debug):
                print(pattern)
            f.write("\n")
            f.write(r'''def LF_Extract_Phrase_Regex_'''+functionName+'''(c):
   
    pattern = \''''+extracts_df.loc[class_names[i]]['complaint words']+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return ('''+str(i)+''',1)
    return (-1,0)''')
            f.write("\n")
    f.close()
    
def create_LF_Description_Regex(debug=False):
    if os.path.exists("Regex_Based_Description_LFs.py"):
        os.remove("Regex_Based_Description_LFs.py")
    f = open("Regex_Based_Description_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
       
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Desc_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+''.join(regexPatterns[class_names[i]])+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return ('''+str(i)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()
    
    
    
def create_LF_Title_Regex(debug=False):
    if os.path.exists("Regex_Based_Title_LFs.py"):
        os.remove("Regex_Based_Title_LFs.py")
    f = open("Regex_Based_Title_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Title_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+ ''.join(regexPatterns[class_names[i]]) +'''\'
    if(re.search(pattern,c['complaint_title'],flags=re.I)):
            return ('''+str(i)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()


def create_LF_Based_On_Embeddings(debug=False):
    if os.path.exists("Embeddings_Based_LFs.py"):
        os.remove("Embeddings_Based_LFs.py")
    f = open("Embeddings_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern, '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
       
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)

        f.write("\n")
        f.write(r'''def LF_Embedding_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    sc = 0
    word_vectors = get_word_vectors(c['complaint_text'])
    for ow in words:
        sc=max(sc,get_similarity(word_vectors,ow))
    return ('''+str(i)+''',sc)''')
        f.write("\n")
    f.close()

def create_LF_Based_On_Embeddings_Title(debug=False):
    if os.path.exists("Title_Embeddings_Based_LFs.py"):
        os.remove("Title_Embeddings_Based_LFs.py")
    f = open("Title_Embeddings_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern, '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
       
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)

        f.write("\n")
        f.write(r'''def LF_Title_Embedding_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    sc = 0
    word_vectors = get_word_vectors(c['complaint_title'].lower().split())
    for ow in words:
        sc=max(sc,get_similarity(word_vectors,ow))
    return ('''+str(i)+''',sc)''')
        f.write("\n")
    f.close()

def create_LF_Based_On_Embeddings_Description(debug=False):
    if os.path.exists("Description_Embeddings_Based_LFs.py"):
        os.remove("Description_Embeddings_Based_LFs.py")
    f = open("Description_Embeddings_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern, '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
       
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)

        f.write("\n")
        f.write(r'''def LF_Description_Embedding_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    sc = 0
    word_vectors = get_word_vectors(c['complaint_description'].lower().split())
    for ow in words:
        sc=max(sc,get_similarity(word_vectors,ow))
    return ('''+str(i)+''',sc)''')
        f.write("\n")
    f.close()


def create_LF_Based_On_TFIDF(debug=False):
    if os.path.exists("TFIDF_Based_LFs.py"):
        os.remove("TFIDF_Based_LFs.py")
    f = open("TFIDF_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])       
               
        f.write("\n")
        f.write(r'''def LF_TFIDF_{}(c):
    sc = cosine_similarity(class_words_tfidf['{}'],tfidf_matrix[c.name+{}])
    return ({},sc)'''.format(functionName,class_names[i],len(class_names),i))
        f.write("\n")
    f.close()



In [34]:
LF_Names = []
LF_output_map = dict()
LF_l = []

create_LF_Title_Regex()
create_LF_Description_Regex()
# create_LF_Based_On_Keywords()
# create_LF_Extracts_Phrases_Regex()
# create_LF_Based_On_Category_Name()
# create_LF_Based_On_Embeddings()
# create_LF_Based_On_TFIDF()
# create_LF_Based_On_Embeddings_Title()
# create_LF_Based_On_Embeddings_Description()



        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Category_Name_'+functionName)
#         LF_output_map['LF_Category_Name_'+functionName]=i
#         LF_l.append(i)
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Embedding_'+functionName)
#         LF_output_map['LF_Embedding_'+functionName]=i
        

        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_TFIDF_'+functionName)
#         LF_output_map['LF_TFIDF_'+functionName]=i

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_KeyWord_'+functionName)
        LF_output_map['LF_KeyWord_'+functionName]=i
        LF_l.append(i)

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Title_Regex_'+functionName)
        LF_output_map['LF_Title_Regex_'+functionName]=i
        LF_l.append(i)

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Desc_Regex_'+functionName)
        LF_output_map['LF_Desc_Regex_'+functionName]=i
        LF_l.append(i)

for i in range(len(class_names)):
    if(class_names[i] in extracts_df.index.tolist()):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Extract_Phrase_Regex_'+functionName)
        LF_output_map['LF_Extract_Phrase_Regex_'+functionName]=i
        LF_l.append(i)
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Title_Embedding_'+functionName)
#         LF_output_map['LF_Title_Embedding_'+functionName]=i
#         LF_l.append(i)
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Description_Embedding_'+functionName)
#         LF_output_map['LF_Description_Embedding_'+functionName]=i
#         LF_l.append(i)
 

        

print('['+','.join(LF_Names)+']')


[LF_KeyWord_AirPollution,LF_KeyWord_AutorickshawsandTaxis,LF_KeyWord_BMTCDriverorConductor,LF_KeyWord_BMTCNeednewBusRoute,LF_KeyWord_BMTCOthers,LF_KeyWord_BadRoads,LF_KeyWord_BrokenStormWaterDrains,LF_KeyWord_Cattle,LF_KeyWord_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_KeyWord_DesiltingLakes,LF_KeyWord_Diseases,LF_KeyWord_Electricity,LF_KeyWord_FloodingofRoadsandFootpaths,LF_KeyWord_Footpaths,LF_KeyWord_Garbage,LF_KeyWord_GovernmentLandEncroachment,LF_KeyWord_HawkersandVendors,LF_KeyWord_Hoardings,LF_KeyWord_IllegalpostersandHoardings,LF_KeyWord_LakesOthers,LF_KeyWord_MaintenanceofRoadsandFootpathsOthers,LF_KeyWord_Manholes,LF_KeyWord_Mosquitos,LF_KeyWord_NeedNewStreetlights,LF_KeyWord_NeedNewToilets,LF_KeyWord_NewBusShelters,LF_KeyWord_NoSewageDrains,LF_KeyWord_NoisePollution,LF_KeyWord_Others,LF_KeyWord_OverflowofStormWaterDrains,LF_KeyWord_ParkingViolations,LF_KeyWord_Parksandplaygrounds,LF_KeyWord_Potholes,LF_KeyWord_PublicNuisan

In [22]:
%load Regex_Based_Title_LFs.py

In [24]:
%load Regex_Based_Description_LFs.py

In [25]:
LFs = [LF_Title_Regex_AirPollution,LF_Title_Regex_AutorickshawsandTaxis,LF_Title_Regex_BMTCDriverorConductor,LF_Title_Regex_BMTCNeednewBusRoute,LF_Title_Regex_BMTCOthers,LF_Title_Regex_BadRoads,LF_Title_Regex_BrokenStormWaterDrains,LF_Title_Regex_Cattle,LF_Title_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Title_Regex_DesiltingLakes,LF_Title_Regex_Diseases,LF_Title_Regex_Electricity,LF_Title_Regex_FloodingofRoadsandFootpaths,LF_Title_Regex_Footpaths,LF_Title_Regex_Garbage,LF_Title_Regex_GovernmentLandEncroachment,LF_Title_Regex_HawkersandVendors,LF_Title_Regex_Hoardings,LF_Title_Regex_IllegalpostersandHoardings,LF_Title_Regex_LakesOthers,LF_Title_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Title_Regex_Manholes,LF_Title_Regex_Mosquitos,LF_Title_Regex_NeedNewStreetlights,LF_Title_Regex_NeedNewToilets,LF_Title_Regex_NewBusShelters,LF_Title_Regex_NoSewageDrains,LF_Title_Regex_NoisePollution,LF_Title_Regex_Others,LF_Title_Regex_OverflowofStormWaterDrains,LF_Title_Regex_ParkingViolations,LF_Title_Regex_Parksandplaygrounds,LF_Title_Regex_Potholes,LF_Title_Regex_PublicNuisance,LF_Title_Regex_Repairofstreetlights,LF_Title_Regex_SewageandStormWaterDrainsOthers,LF_Title_Regex_StrayDogs,LF_Title_Regex_Traffic,LF_Title_Regex_TreesParksandPlaygroundsOthers,LF_Title_Regex_UnauthorizedConstruction,LF_Title_Regex_WaterLeakage,LF_Title_Regex_WaterSupply,LF_Desc_Regex_AirPollution,LF_Desc_Regex_AutorickshawsandTaxis,LF_Desc_Regex_BMTCDriverorConductor,LF_Desc_Regex_BMTCNeednewBusRoute,LF_Desc_Regex_BMTCOthers,LF_Desc_Regex_BadRoads,LF_Desc_Regex_BrokenStormWaterDrains,LF_Desc_Regex_Cattle,LF_Desc_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Desc_Regex_DesiltingLakes,LF_Desc_Regex_Diseases,LF_Desc_Regex_Electricity,LF_Desc_Regex_FloodingofRoadsandFootpaths,LF_Desc_Regex_Footpaths,LF_Desc_Regex_Garbage,LF_Desc_Regex_GovernmentLandEncroachment,LF_Desc_Regex_HawkersandVendors,LF_Desc_Regex_Hoardings,LF_Desc_Regex_IllegalpostersandHoardings,LF_Desc_Regex_LakesOthers,LF_Desc_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Desc_Regex_Manholes,LF_Desc_Regex_Mosquitos,LF_Desc_Regex_NeedNewStreetlights,LF_Desc_Regex_NeedNewToilets,LF_Desc_Regex_NewBusShelters,LF_Desc_Regex_NoSewageDrains,LF_Desc_Regex_NoisePollution,LF_Desc_Regex_Others,LF_Desc_Regex_OverflowofStormWaterDrains,LF_Desc_Regex_ParkingViolations,LF_Desc_Regex_Parksandplaygrounds,LF_Desc_Regex_Potholes,LF_Desc_Regex_PublicNuisance,LF_Desc_Regex_Repairofstreetlights,LF_Desc_Regex_SewageandStormWaterDrainsOthers,LF_Desc_Regex_StrayDogs,LF_Desc_Regex_Traffic,LF_Desc_Regex_TreesParksandPlaygroundsOthers,LF_Desc_Regex_UnauthorizedConstruction,LF_Desc_Regex_WaterLeakage,LF_Desc_Regex_WaterSupply]



In [35]:
# pkl.dump(LF_Names,open("LF_Names_245.p","wb"))
# pkl.dump(LF_output_map,open("LF_output_map_245.p","wb"))
# pkl.dump(LF_l,open("LF_l_245.p","wb"))

pkl.dump(LF_Names,open("LF_Names_161.p","wb"))
pkl.dump(LF_output_map,open("LF_output_map_161.p","wb"))
pkl.dump(LF_l,open("LF_l_161.p","wb"))
print(len(LF_Names),len(LF_output_map),len(LF_l))


161 161 161


In [27]:
def get_L_S_Tensor(df,msg):     
    L_S = []
    print('labelling ',msg,' data')
    for i in range(len(df.index)):
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
#             print(i,LF.__name__)            
            l,s = LF(df.iloc[i])
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
        
    return L_S


import time
import datetime
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

# val_L_S = get_L_S_Tensor(val_df)

# pkl.dump(val_L_S,open("val_L_S.p","wb"))

# test_L_S = get_L_S_Tensor(test_df,'test')
# pkl.dump(test_L_S,open("test_L_S_TFIDF.p","wb"))

# train_L_S = get_L_S_Tensor(train_df,'train')
# pkl.dump(train_L_S,open("train_L_S_TFIDF.p","wb"))


# train_L_S = get_L_S_Tensor(train_df,'keywords train')
# pkl.dump(train_L_S,open("train_L_S_Keywords.p","wb"))

# test_L_S = get_L_S_Tensor(test_df,'keywords test')
# pkl.dump(test_L_S,open("test_L_S_Keywords_regex_extracts.p","wb"))

# train_L_S = get_L_S_Tensor(train_df,'keywords train')
# pkl.dump(train_L_S,open("train_L_S_Keywords_regex_extracts.p","wb"))

test_L_S = get_L_S_Tensor(test_df,'regex test')
pkl.dump(test_L_S,open("test_L_S_regex.p","wb"))

train_L_S = get_L_S_Tensor(train_df,'regex train')
pkl.dump(train_L_S,open("train_L_S_regex.p","wb"))


# test_L_S = get_L_S_Tensor(test_df,'embeddings test')
# pkl.dump(test_L_S,open("test_L_S_Embeddings.p","wb"))

# test_L_S = get_L_S_Tensor(test_df,'T and D embeddings test')
# pkl.dump(test_L_S,open("test_L_S_T-D_Embeddings.p","wb"))



# train_L_S = get_L_S_Tensor(train_df,'embeddings train')
# pkl.dump(train_L_S,open("train_L_S_Embeddings.p","wb"))

# train_L_S = get_L_S_Tensor(train_df,'T and D embeddings train')
# pkl.dump(train_L_S,open("train_L_S_T-D_Embeddings.p","wb"))

# gold_L_S = get_L_S_Tensor(gold_df,'gold-labels-clean')
# pkl.dump(gold_L_S,open("gold-labels-clean.p","wb"))


print(str(datetime.timedelta(seconds=time.time() - start_time)))



started at: 14-5-2018, 17:34:3
labelling  keywords test  data
500data points labelled in 0.0825196623802185 mins
1000data points labelled in 0.1646123170852661 mins
1500data points labelled in 0.24780991474787395 mins
2000data points labelled in 0.33310418923695884 mins
2500data points labelled in 0.4195414463678996 mins
3000data points labelled in 0.5027475953102112 mins
3500data points labelled in 0.5834892471631368 mins
4000data points labelled in 0.6665494680404663 mins
4500data points labelled in 0.7496981739997863 mins
5000data points labelled in 0.8299444238344829 mins
5500data points labelled in 0.9106906294822693 mins
6000data points labelled in 0.9915045777956645 mins
6500data points labelled in 1.0725292364756267 mins
labelling  regex test  data
500data points labelled in 1.1809990525245666 mins
1000data points labelled in 1.2610489487648011 mins
1500data points labelled in 1.3414787213007608 mins
2000data points labelled in 1.422957138220469 mins
2500data points labelled in

In [325]:
# import sys
# !{sys.executable} -m pip install tensorflow
!type python

python is /usr/bin/python


In [319]:


pkl.dump(val_L_S,open("val_L_S.p","wb"),protocol=2)
pkl.dump(train_L_S,open("train_L_S.p","wb"),protocol=2)
pkl.dump(test_L_S,open("test_L_S.p","wb"),protocol=2)


In [618]:
import numpy as np
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def process(s):
    return ''.join([i for i in s if not i.isdigit() and i not in '_' ]).lower()
    
vectorizer = TfidfVectorizer(preprocessor=process)
tfidf_matrix = vectorizer.fit_transform(class_words+train_df['complaint_description'].tolist())


In [655]:
print(len(vectorizer.vocabulary_))
idf = vectorizer.idf_
d = dict(zip(vectorizer.get_feature_names(), idf))

from collections import OrderedDict
d = OrderedDict(sorted(d.items(),key=lambda kv: kv[1], reverse=True))
# for x in d.items():
#     print(x)
for c in class_names:
    for w in re.sub('-|,','',c.lower()).split():
        if w in d.keys():
            print(w,d[w])
        else:
            print(w,'not in vocabulary')

31917
air 5.655315363914983
pollution 5.31884312729377
autorickshaws 8.397851741088363
and 1.5476140735768542
taxis 7.731372807610579
bmtc 4.515745496698209
driver 5.912945091300362
or 3.328616577422118
conductor 6.798995937984799
bmtc 4.515745496698209
need 3.961411684690687
new 4.3365017010013
bus 3.5158475847987893
route 4.549946292783993
bmtc 4.515745496698209
others 6.090017292208173
bad 3.5015842685054035
roads 3.1803449207861125
broken 4.996654359426207
storm 5.615442972667606
water 3.3725871369761475
drains 5.31884312729377
cattle 7.065624601238747
clearing 6.588700529148438
of 1.6403075439294033
blockage 6.717317906970532
of 1.6403075439294033
under 4.774212808842828
ground 5.10984270370428
drainage 4.429547899761626
pipelines 7.316939029519654
and 1.5476140735768542
replacement 8.164236889906856
of 1.6403075439294033
damaged 4.847025164317795
or 3.328616577422118
missing 6.777942528786967
manhole 6.154904914414048
cover 5.882705206110644
desilting 8.703233390639543
lakes 7.49

In [620]:

# feature_names = vectorizer.get_feature_names()
# print(len(feature_names))
# print((feature_names))


class_words_tfidf= dict()
for i,cat in enumerate(class_names):
    class_words_tfidf[cat] = tfidf_matrix.getrow(i)


In [21]:

def LF_KeyWord_AirPollution(c):
    words = ["air","pollution"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (0,1)
    return (-1,0)

def LF_KeyWord_AutorickshawsandTaxis(c):
    words = ["autorickshaws","taxis","taxi","auto","autorickshaw"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (1,1)
    return (-1,0)

def LF_KeyWord_BMTCDriverorConductor(c):
    words = ["bmtc","driver","conductor"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (2,1)
    return (-1,0)

def LF_KeyWord_BMTCNeednewBusRoute(c):
    words = ["bmtc","need","new","bus","route"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (3,1)
    return (-1,0)

def LF_KeyWord_BMTCOthers(c):
    words = ["bmtc"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (4,1)
    return (-1,0)

def LF_KeyWord_BadRoads(c):
    words = ["bad","roads","road"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (5,1)
    return (-1,0)

def LF_KeyWord_BrokenStormWaterDrains(c):
    words = ["broken","storm","water","drains","overflow","drainage"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (6,1)
    return (-1,0)

def LF_KeyWord_Cattle(c):
    words = ["cattle","cows","buffaloes","goats"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (7,1)
    return (-1,0)

def LF_KeyWord_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover(c):
    words = ["clearing","blockage","under","ground","drainage","pipelines","replacement","damaged","missing","manhole","cover"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (8,1)
    return (-1,0)

def LF_KeyWord_DesiltingLakes(c):
    words = ["desilting","lakes"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (9,1)
    return (-1,0)

def LF_KeyWord_Diseases(c):
    words = ["diseases","malaria","dengue","cholera"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (10,1)
    return (-1,0)

def LF_KeyWord_Electricity(c):
    words = ["electricity","power","current","power cut"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (11,1)
    return (-1,0)

def LF_KeyWord_FloodingofRoadsandFootpaths(c):
    words = ["flooding","roads","footpaths","water","flood","floods"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (12,1)
    return (-1,0)

def LF_KeyWord_Footpaths(c):
    words = ["footpaths","footpath"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (13,1)
    return (-1,0)

def LF_KeyWord_Garbage(c):
    words = ["garbage","waste","plastic","dirt"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (14,1)
    return (-1,0)

def LF_KeyWord_GovernmentLandEncroachment(c):
    words = ["government","land","encroachment","occupy","illegal"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (15,1)
    return (-1,0)

def LF_KeyWord_HawkersandVendors(c):
    words = ["hawkers","vendors"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (16,1)
    return (-1,0)

def LF_KeyWord_Hoardings(c):
    words = ["hoardings","advertisements"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (17,1)
    return (-1,0)

def LF_KeyWord_IllegalpostersandHoardings(c):
    words = ["illegal","posters","hoardings"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (18,1)
    return (-1,0)

def LF_KeyWord_LakesOthers(c):
    words = ["lakes","lake"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (19,1)
    return (-1,0)

def LF_KeyWord_MaintenanceofRoadsandFootpathsOthers(c):
    words = ["maintenance","roads","footpaths"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (20,1)
    return (-1,0)

def LF_KeyWord_Manholes(c):
    words = ["manholes","manhole","man hole"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (21,1)
    return (-1,0)

def LF_KeyWord_Mosquitos(c):
    words = ["mosquitos","mosquitoes"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (22,1)
    return (-1,0)

def LF_KeyWord_NeedNewStreetlights(c):
    words = ["need","new","streetlights","streetlight","light","new streetlight"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (23,1)
    return (-1,0)

def LF_KeyWord_NeedNewToilets(c):
    words = ["need","new","toilets","toilet"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (24,1)
    return (-1,0)

def LF_KeyWord_NewBusShelters(c):
    words = ["new","bus","shelters","shelter"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (25,1)
    return (-1,0)

def LF_KeyWord_NoSewageDrains(c):
    words = ["sewage","drains","drainage"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (26,1)
    return (-1,0)

def LF_KeyWord_NoisePollution(c):
    words = ["noise","pollution","siren","speakers","speakers","loud"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (27,1)
    return (-1,0)

def LF_KeyWord_Others(c):
    words = ["others"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (28,1)
    return (-1,0)

def LF_KeyWord_OverflowofStormWaterDrains(c):
    words = ["overflow","storm","water","drains","pipes"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (29,1)
    return (-1,0)

def LF_KeyWord_ParkingViolations(c):
    words = ["parking","violations"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (30,1)
    return (-1,0)

def LF_KeyWord_Parksandplaygrounds(c):
    words = ["parks","playgrounds","park","play","playground"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (31,1)
    return (-1,0)

def LF_KeyWord_Potholes(c):
    words = ["potholes","holes","pothole"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (32,1)
    return (-1,0)

def LF_KeyWord_PublicNuisance(c):
    words = ["public","nuisance"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (33,1)
    return (-1,0)

def LF_KeyWord_Repairofstreetlights(c):
    words = ["repair","streetlights","streetlight","light","broken","damaged"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (34,1)
    return (-1,0)

def LF_KeyWord_SewageandStormWaterDrainsOthers(c):
    words = ["sewage","storm","water","drains","drainage"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (35,1)
    return (-1,0)

def LF_KeyWord_StrayDogs(c):
    words = ["stray","dogs","dog"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (36,1)
    return (-1,0)

def LF_KeyWord_Traffic(c):
    words = ["traffic","vehicles"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (37,1)
    return (-1,0)

def LF_KeyWord_TreesParksandPlaygroundsOthers(c):
    words = ["trees","parks","playgrounds","tree"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (38,1)
    return (-1,0)

def LF_KeyWord_UnauthorizedConstruction(c):
    words = ["unauthorized","construction"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (39,1)
    return (-1,0)

def LF_KeyWord_WaterLeakage(c):
    words = ["water","leakage"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (40,1)
    return (-1,0)

def LF_KeyWord_WaterSupply(c):
    words = ["water","supply"]
    if(len(set(c['complaint_text']).intersection(words))>0):
            return (41,1)
    return (-1,0)
